## Discipline of AI Checkpoint 01
develop study of data of post's twitter's influence in price of BTC.
### Step one - Pre-processing of data
Nesta primeira etapa do projeto, o aluno deve escrever um relatório descrevendo a aplicação que vai ser utilizada nesta disciplina. As seguintes informações devem estar presentes no relatório.
1. PROBLEMA:  Descobrir no periodo de 10 de fevereiro de 2021 a 15 de agosto de 2021 se os posts do Twitter sobre Biticoin influenciaram no valor da cotação da moeda, analisando o sentimento do texto de cada post.

2. DADOS:
  - Quantos atributos, instancias e classes da base escolhida possui? possui 6 atributos, 1 milhão de instâncias e uma classe.

  - Quantos atributos categóricos e numéricos a base possui? 
    Numericos são "price", "positive", "negative", "neutral", "compound" e categoríco o atributo "sentiment"

  - Para os atributos categóricos, quantas classes existem para cada atributo? 
    Existem 3 classe possíveis, 'Positive', 'Neutral' e 'Negative'.

  - Para os atributos numéricos, são todos contínuos? Se sim, quais os intervalos de valores? 
    Sim, tem atributos numéricos contínuos, que vão de -1 a 1 para os 4 atributos do nivel de intensidade dos sentimentos, e de 44,836.0 a 46,991.3 para o preço do bitcoin.

3. Pré-processamento:
  - Foi utilizado dois datasets, um com preço e data da cotação do bitcoin e outra com os twitters.
  - Foi feita uma limpeza para remover alguns dados onde tinha dados de data inválidos.
  - Foi feita a correção da data da base 01 e a base 02 para que fossem normalizados com o padrão mês, dia e ano.
  - Foi criada uma coluna "price" para onde na base 01 a data fosse igual a base 02 para ter o preço do dia do post no twitter.
  - Foi limitado a 1 milhão de instâncias da base 01.
  - Para transforma o texto de cada twitter em um vetor de palavras:
    - Foi usado o algoritmo Monolingual Word Alignment (MWA) para organizar os dados para passar no lematizador.
    - Foi utilizado um lematizador de palavras baseado no algoritmo de lematização de Porter.
    - Foi definido um atributo no dataset cleantext onde contem o vetor de palavras.
  - Para criar a coluna classes de sentimentos:
    - Foi analisado usando o SentimentIntensityAnalyzer do conjunto nltk.
    - Foi definido so seguintes parametros para cada classe de sentimento: positivo (compound >= 0.05), neutro (compound > -0.05 e compound < 0.05) e negativo (compound < -0.05).

OBS: Sobre o coumpound, quando um valor fica próximo de 1 indica um sentimento positivo, se um valor está próximo de -1 indica um sentimento negativo e um valor próximo de 0 indica um sentimento neutro.
O cálculo específico do valor composto pode envolver fórmulas matemáticas que levam em consideração a distribuição das pontuações, bem como os pesos atribuídos a diferentes aspectos das palavras e do contexto. Esses detalhes são geralmente tratados no código da implementação do VADER, que pode variar dependendo da linguagem de programação utilizada e das configurações específicas.

In [14]:
import pandas as pd
from tqdm import tqdm

#For Preprocessing
import re    # RegEx for removing non-letter characters
import nltk  # natural language processing
nltk.download("stopwords")
nltk.downloader.download('vader_lexicon')
from nltk.corpus import stopwords
from nltk.stem.porter import *
from nltk.sentiment.vader import SentimentIntensityAnalyzer

pd.options.plotting.backend = "plotly"


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\joelm\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\joelm\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


# Load Tweet dataset
dataset price bitcoin: https://www.investing.com/crypto/bitcoin/historical-data
dataset twitter: https://www.kaggle.com/datasets/kaushiksuresh147/bitcoin-tweets

In [45]:
# Load Tweet dataset
df1 = pd.read_csv('../data/raw/Bitcoin_tweets.csv', nrows=1000000)
df2 = pd.read_csv('../data/raw/BitcoinPriceHistory.csv')


#'date']=="['ETH', 'BTC', 'Bitcoin']"
df1=df1.drop(df1.index[64943])
df1=df1.drop(df1.index[137067])
df1=df1.drop(df1.index[180573])
df1=df1.drop(df1.index[693191])
df1=df1.drop(df1.index[697393])

df2['Date'] = pd.to_datetime(df2['Date'], format='%m/%d/%Y')
df2['Date'] = df2['Date'].dt.tz_localize('UTC').dt.strftime('%m-%d-%Y')

df1['date'] = pd.to_datetime(df1['date'])
df1['date'] = df1['date'].dt.tz_localize('UTC').dt.strftime('%m-%d-%Y')

df1['price'] = df1['date'].map(df2.set_index('Date')['Price'])
# Validate if the data equals a certain value
validation_value = 'expected_value'
mask = df1['price'] == validation_value

if mask.all():
    print('Data equals the expected value')
else:
    print('Data does not equal the expected value')

df = df1[['text', 'price', 'date']][:1000000]
df


C:\Users\joelm\AppData\Local\Temp\ipykernel_8576\2768624893.py:2: DtypeWarning: Columns (5,6,7,12) have mixed types. Specify dtype option on import or set low_memory=False.
  df1 = pd.read_csv('../data/raw/Bitcoin_tweets.csv', nrows=1000000)


Data does not equal the expected value


,text,price,date
0,Blue Ridge Bank shares halted by NYSE after #b...,"44,836.0",02-10-2021
1,"😎 Today, that's this #Thursday, we will do a ""...","44,836.0",02-10-2021
2,"Guys evening, I have read this article about B...","44,836.0",02-10-2021
3,$BTC A big chance in a billion! Price: \487264...,"44,836.0",02-10-2021
4,This network is secured by 9 508 nodes as of t...,"44,836.0",02-10-2021
...,...,...,...
999995,#wtfhappenedin1971? \n#Bitcoin will fix this!,"46,991.3",08-15-2021
999996,@CryptoNadine @BabyArabiaBSC #babyarabia massi...,"46,991.3",08-15-2021
999997,@mcuban Are you serious or part of the scam??\...,"46,991.3",08-15-2021
999998,Why is #Bitcoin going to ascend to a global re...,"46,991.3",08-15-2021


In [46]:
def tweet_to_words(tweet):
    ''' Convert tweet text into a sequence of words '''
    
    # convert to lowercase
    text = tweet.lower()
    # remove non letters
    text = re.sub(r"[^a-zA-Z0-9]", " ", text)
    # tokenize
    words = text.split()
    # remove stopwords
    words = [w for w in words if w not in stopwords.words("english")]
    # apply stemming
    words = [PorterStemmer().stem(w) for w in words]
    # return list
    return words

In [47]:
cleantext=[]
for item in tqdm(df['text']):
    words=tweet_to_words(item)
    cleantext+=[words]
df['cleantext']=cleantext
df

100%|██████████| 100000/100000 [04:53<00:00, 340.81it/s]


,text,price,date,cleantext
0,Blue Ridge Bank shares halted by NYSE after #b...,"44,836.0",02-10-2021,"[blue, ridg, bank, share, halt, nyse, bitcoin,..."
1,"😎 Today, that's this #Thursday, we will do a ""...","44,836.0",02-10-2021,"[today, thursday, take, 2, friend, leowandersl..."
2,"Guys evening, I have read this article about B...","44,836.0",02-10-2021,"[guy, even, read, articl, btc, would, like, sh..."
3,$BTC A big chance in a billion! Price: \487264...,"44,836.0",02-10-2021,"[btc, big, chanc, billion, price, 4872644, 0, ..."
4,This network is secured by 9 508 nodes as of t...,"44,836.0",02-10-2021,"[network, secur, 9, 508, node, today, soon, bi..."
...,...,...,...,...
99996,@NftCryptosArt A #legendary #NFT item from \n@...,"55,646.1",04-19-2021,"[nftcryptosart, legendari, nft, item, neondist..."
99997,@SnowGotMelted @NFTE @NFTCollector11 @KennethB...,"55,646.1",04-19-2021,"[snowgotmelt, nfte, nftcollector11, kennethbos..."
99998,@NftPasso A #legendary #NFT item from \n@neond...,"55,646.1",04-19-2021,"[nftpasso, legendari, nft, item, neondistrictr..."
99999,My Crypto Strategy on ICONOMI just outperforme...,"55,646.1",04-19-2021,"[crypto, strategi, iconomi, outperform, btc, c..."


In [48]:
def unlist(list):
    return ''.join(f'{item} ' for item in list)

In [49]:
class SentimentAnalyzer:
    def __init__(self):
        self.sentiment_detector = SentimentIntensityAnalyzer()

    def analyze_sentiment(self, text):
        sentiment_scores = self.sentiment_detector.polarity_scores(text)
        if sentiment_scores['compound'] >= 0.05:
            sentiment_scores['sentiment'] = 'Positive'
        elif -0.05 < sentiment_scores['compound'] < 0.05:
            sentiment_scores['sentiment'] = 'Neutral'
        else:
            sentiment_scores['sentiment'] = 'Negative'
        return sentiment_scores

In [61]:
def compute_vader_scores(df, label):
    sid = SentimentIntensityAnalyzer()
    analyzer = SentimentAnalyzer()
    
    df["negative"] = df[label].apply(lambda x: sid.polarity_scores(unlist(x))["neg"])
    df["neutral"] = df[label].apply(lambda x: sid.polarity_scores(unlist(x))["neu"])
    df["positive"] = df[label].apply(lambda x: sid.polarity_scores(unlist(x))["pos"])
    df["compound"] = df[label].apply(lambda x: sid.polarity_scores(unlist(x))["compound"])
    df['cleantext2'] = df[label].apply(lambda x: unlist(x))
    df['sentiment'] = df[label].apply(lambda x: analyzer.analyze_sentiment(unlist(x))["sentiment"])
    return df

In [63]:

df = compute_vader_scores(df,'cleantext')
df
df[['price', 'positive', 'negative', 'neutral', 'compound', 'sentiment']]

,price,positive,negative,neutral,compound,sentiment
0,"44,836.0",0.128,0.0,0.872,0.2960,Positive
1,"44,836.0",0.225,0.0,0.775,0.4939,Positive
2,"44,836.0",0.281,0.0,0.719,0.5719,Positive
3,"44,836.0",0.000,0.0,1.000,0.0000,Neutral
4,"44,836.0",0.000,0.2,0.800,-0.5423,Negative
...,...,...,...,...,...,...
99996,"55,646.1",0.000,0.0,1.000,0.0000,Neutral
99997,"55,646.1",0.000,0.0,1.000,0.0000,Neutral
99998,"55,646.1",0.000,0.0,1.000,0.0000,Neutral
99999,"55,646.1",0.000,0.0,1.000,0.0000,Neutral
